# 전환율을 이용해서 실질월세 계산

In [5]:
import pandas as pd
from APICall import HouseType

In [6]:
def calc_newRent(houseType: HouseType, year: int) -> pd.DataFrame:
    FileName = f"data/{houseType.value}_{year}.csv"
    df = pd.read_csv(FileName)

    ChangeRateFileName = f"data/ChangeRate_SDMG_{houseType.value}.csv"
    ChangeRate = pd.read_csv(ChangeRateFileName)

    df["YM"] = df["dealDate"] // 100
    merged = pd.merge(df, ChangeRate, left_on="YM", right_on="Date", how="left")

    merged["monthlyRent"] = merged["monthlyRent"].astype(str).str.replace(',', '').astype(float)
    merged["deposit"] = merged["deposit"].astype(str).str.replace(',', '').astype(float)
    merged["calcedRent"] = merged["monthlyRent"] + merged["deposit"]*merged["ChangeRate"]/1200
    
    merged["RPA"] = merged["calcedRent"] / merged["excluUseAr"]
    merged["RPA"] = merged["RPA"].round(2)

    merged = merged.drop(columns=["YM", "Date"])
    columns = ["dealDate", "deposit", "monthlyRent", "ChangeRate", "calcedRent", "excluUseAr", "RPA"]
    for col in merged.columns:
        if col not in columns:
            columns.append(col)
    merged = merged[columns]
    print(merged)
    merged.to_csv("data/tmp.csv")
    

In [7]:
calc_newRent(HouseType.Apt, 2010)

      dealDate  deposit  monthlyRent  ChangeRate  calcedRent  excluUseAr  \
0     20110105   1500.0         60.0     7.20000   69.000000      68.760   
1     20110105  18000.0          0.0     7.20000  108.000000      59.584   
2     20110110  22000.0          0.0     7.20000  132.000000      84.740   
3     20110110   7000.0        100.0     7.20000  142.000000      84.240   
4     20110120  28000.0          0.0     7.20000  168.000000     122.910   
...        ...      ...          ...         ...         ...         ...   
1075  20191220  56000.0          0.0     3.71594  173.410533      84.971   
1076  20191224  43000.0          0.0     3.71594  133.154517      84.330   
1077  20191225  32500.0          0.0     3.71594  100.640042      59.400   
1078  20191230  50000.0          0.0     3.71594  154.830833      84.956   
1079  20191230  90000.0          0.0     3.71594  278.695500     120.492   

       RPA        aptNm  buildYear contractTerm contractType  floor   jibun  \
0     1.